In [126]:
import pandas as pd
import pickle
import numpy as np
import sys
sys.path.append("../")
from src.soporte_transformers import (
    load_models
)

In [127]:
df = pd.read_csv("../datos/06_rent_outliers_gestionados.csv")
df.sample()

,size,bathrooms,municipality,district,distancia_centro,price,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,...,floor_primero,floor_quinto,floor_segundo,floor_septimo,floor_sexto,floor_sotano,floor_tercero,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
78,-0.207407,0.0,0.619147,0.084038,0.046766,650.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Creamos un dato nuevo
- Para probar nuestras predicciones

In [128]:
# Datos de una nueva casa para predicción
new_house = pd.DataFrame({
    'propertyType': ['flat'],  # Nueva categoría no vista
    'size': [85],
    'rooms': ["2 habitaciones"],
    'bathrooms': ["1 aseo"],
    'floor': ["desconocido"],
    'municipality': ['Madrid'],
    'district': ["Puente de Vallecas"],
    'hasLift' : 'tiene ascensor',
    'distancia_centro' : 'Entre 1 y 5 km'
})

df_new = pd.DataFrame(new_house)
df_new

,propertyType,size,rooms,bathrooms,floor,municipality,district,hasLift,distancia_centro
0,flat,85,2 habitaciones,1 aseo,desconocido,Madrid,Puente de Vallecas,tiene ascensor,Entre 1 y 5 km


In [129]:
target_encoder, one_hot_encoder, robust_scaler, modelo_xgb =  load_models()

In [130]:
df_pred = df_new.copy()
df_pred

,propertyType,size,rooms,bathrooms,floor,municipality,district,hasLift,distancia_centro
0,flat,85,2 habitaciones,1 aseo,desconocido,Madrid,Puente de Vallecas,tiene ascensor,Entre 1 y 5 km


# Separar las columnas por el tipo de encoding que necesitan

In [131]:
cols_target = ["bathrooms","municipality","district","distancia_centro"]
cols_onehot = ["propertyType","rooms","floor","hasLift"]

### Hacer one_hot_encoder

In [132]:
test = one_hot_encoder.transform(df_pred[cols_onehot])
# Obtenemos los nombres de las columnas del codificador
column_names = one_hot_encoder.get_feature_names_out(cols_onehot)
# Convertimos a un DataFrame
onehot_df = pd.DataFrame(test.toarray(), columns=column_names)
onehot_df

,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio,rooms_1 habitacion,rooms_2 habitaciones,rooms_3 habitaciones,rooms_4 habitaciones,...,floor_primero,floor_quinto,floor_segundo,floor_septimo,floor_sexto,floor_sotano,floor_tercero,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Hacer target_encoder

In [133]:
df_pred["precio"] = np.nan
df_pred = target_encoder.transform(df_pred)
df_pred

,propertyType,size,rooms,bathrooms,floor,municipality,district,hasLift,distancia_centro,precio
0,flat,85,2 habitaciones,677.756303,desconocido,705.594108,684.0437,tiene ascensor,701.110187,NaN


### Eliminar las columnas de onehot y unir el df que habíamos generado previamente

In [134]:
df_pred.drop(columns= cols_onehot,inplace=True)
df_pred = pd.concat([df_pred, onehot_df], axis=1)
df_pred

,size,bathrooms,municipality,district,distancia_centro,precio,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,...,floor_primero,floor_quinto,floor_segundo,floor_septimo,floor_sexto,floor_sotano,floor_tercero,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
0,85,677.756303,705.594108,684.0437,701.110187,NaN,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Feature- Scaling

In [135]:
cols_escalar = ["size","bathrooms","municipality","district","distancia_centro"]


In [136]:
df_pred[cols_escalar] = robust_scaler.transform(df_pred[cols_escalar])

In [137]:
df_pred

,size,bathrooms,municipality,district,distancia_centro,precio,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,...,floor_primero,floor_quinto,floor_segundo,floor_septimo,floor_sexto,floor_sotano,floor_tercero,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
0,0.681481,0.0,0.619147,0.15483,0.020011,NaN,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [138]:
df_pred.drop(columns="precio",inplace=True)
modelo_xgb.predict(df_pred)

array([718.37396], dtype=float32)

In [139]:
target_encoder, one_hot_encoder, robust_scaler, modelo_xgb =  load_models()

cols_target = ["bathrooms","municipality","district","distancia_centro"]
cols_onehot = ["propertyType","rooms","floor","hasLift"]
cols_escalar = ["size","bathrooms","municipality","district","distancia_centro"]

# Datos de una nueva casa para predicción
new_house = pd.DataFrame({
    'propertyType': ['flat'],  # Nueva categoría no vista
    'size': [85],
    'rooms': ["2 habitaciones"],
    'bathrooms': ["1 aseo"],
    'floor': ["desconocido"],
    'municipality': ['Madrid'],
    'district': ["Puente de Vallecas"],
    'hasLift' : 'tiene ascensor',
    'distancia_centro' : 'Entre 1 y 5 km'
})

df_new = pd.DataFrame(new_house)
df_pred = df_new.copy()

# Hacemos el OneHot Encoder
onehot = one_hot_encoder.transform(df_pred[cols_onehot])
# Obtenemos los nombres de las columnas del codificador
column_names = one_hot_encoder.get_feature_names_out(cols_onehot)
# Convertimos a un DataFrame
onehot_df = pd.DataFrame(onehot.toarray(), columns=column_names)

# Realizamos el target encoder
df_pred["precio"] = np.nan
df_pred = target_encoder.transform(df_pred)

# Quitamos las columnas que ya han sido onehoteadas 
df_pred.drop(columns= cols_onehot,inplace=True)
df_pred = pd.concat([df_pred, onehot_df], axis=1)

# Escalamos los valores
df_pred[cols_escalar] = robust_scaler.transform(df_pred[cols_escalar])

# Realizamos la predicción
df_pred.drop(columns="precio",inplace=True)
modelo_xgb.predict(df_pred)

array([718.37396], dtype=float32)

# Funciona!


# Ver si puedo guardar listas en pkl


In [140]:
df = pd.read_csv("../datos/03_api_rent_sin_nulos.csv")

In [141]:
df.sample()

,propertyType,price,size,rooms,bathrooms,floor,district,municipality,hasLift,distancia_centro
290,flat,730.0,35.0,1 habitacion,1 aseo,segundo,Centro,Madrid,tiene ascensor,Menos de 1 km


In [142]:
lista_tipo_propiedad = df["propertyType"].unique().tolist()
lista_habitaciones = df["rooms"].unique().tolist()
lista_aseos = df["bathrooms"].unique().tolist()
lista_floor = df["floor"].unique().tolist()
lista_municipality = df["municipality"].unique().tolist()
lista_district = df["district"].unique().tolist()
lista_ascensor = df["hasLift"].unique().tolist()
lista_distancia = df["distancia_centro"].unique().tolist()


In [143]:
lista_distancia = ['Menos de 1 km', 'Entre 1 y 5 km', 'Entre 5 y 10 km', 'Entre 10 y 20 km', 'Entre 20 y 30 km', 'Entre 30 y 40 km', 'Entre 40 y 50 km', 'Mas de 50 km']

In [144]:
lista_floor = ['sotano',"bajo","entreplanta",'primero',"segundo","tercero","cuarto","quinto","sexto","septimo","octavo","decimo cuarto","desconocido"]

### Guardarlos en un pickle

In [145]:
with open('../datos/lista_opciones/01_tipo_propiedad.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_tipo_propiedad), archivo)
with open('../datos/lista_opciones/02_num_habitaciones.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_habitaciones), archivo)
with open('../datos/lista_opciones/03_num_aseos.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_aseos), archivo)
with open('../datos/lista_opciones/04_floor.pkl', 'wb') as archivo:
    pickle.dump(lista_floor, archivo)
with open('../datos/lista_opciones/05_municipio.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_municipality), archivo)
with open('../datos/lista_opciones/06_distrito.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_district), archivo)
with open('../datos/lista_opciones/07_ascensor.pkl', 'wb') as archivo:
    pickle.dump(sorted(lista_ascensor), archivo)
with open('../datos/lista_opciones/08_distancia_centro.pkl', 'wb') as archivo:
    pickle.dump(lista_distancia, archivo)

In [146]:
with open('../datos/lista_opciones/01_tipo_propiedad.pkl', 'rb') as archivo:
    tipo_propiedad = pickle.load(archivo)

In [147]:
tipo_propiedad

['chalet', 'countryHouse', 'duplex', 'flat', 'penthouse', 'studio']